# How-to: Qiskit Hybrid Job

## Prepare files

### Prepare Hybrid job script

In [1]:
! cat ./data/3_hybrid_jobs/algorithm_script.py

"""Example of usage of Qiskit-Braket provider."""

from qiskit import QuantumCircuit

from braket.jobs import save_job_result
from qiskit_braket_provider import BraketProvider

provider = BraketProvider()
backend = provider.get_backend("SV1")
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)

results = backend.run(circuit, shots=1)

print(results.result().get_counts())
save_job_result(results.result().get_counts())


### Prepare docker image

In [2]:
! cat ./data/3_hybrid_jobs/Dockerfile

FROM 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-base-jobs:1.0-cpu-py37-ubuntu18.04

RUN python3 -m pip install --upgrade pip

RUN python3 -m pip install --no-cache --upgrade git+https://github.com/qiskit-community/qiskit-braket-provider


## Prepare env

### Login to AWS CLI and docker

Login to AWS' ERC to pull base image

`aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 292282985366.dkr.ecr.us-west-2.amazonaws.com`

Login to yours ERC to push your image

`aws ecr get-login-password --region <YOUR_AWS_REGION> | docker login --username AWS --password-stdin <YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_AWS_REGION>.amazonaws.com`

### Create ECR repo

`! aws ecr create-repository --repository-name amazon-braket-qiskit-provider`

### Build docker image

`! docker build -t amazon-braket-qiskit-provider ./data/2_hybrid_jobs/`

`! docker tag amazon-braket-qiskit-provider:latest <YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_AWS_REGION>.amazonaws.com/amazon-braket-qiskit-provider:latest`

## Running your job

In [ ]:
import time
from pprint import pprint

from braket.aws import AwsQuantumJob

image_uri = "<ACCOUNT_ID>.dkr.ecr.<AWS_REGION>.amazonaws.com/amazon-braket-qiskit-provider:latest"

job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="data/3_hybrid_jobs/algorithm_script.py",
    wait_until_complete=False,
    job_name="qiskit-job-0",
    image_uri=image_uri,
)

In [ ]:
pprint(job.arn)

while job.state() not in AwsQuantumJob.TERMINAL_STATES:
    pprint(job.state())
    time.sleep(20)

pprint(job.result())